<a href="https://colab.research.google.com/github/biplobsd/stellar-claimable/blob/main/Claimable_sends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# t.me/biplob_sd | biplobsd.github.io
#@title Claimable sends (Stellar)
SECRET_KEY = "" #@param {type:"string"}
ACCOUNT_CSV_PATH = "" #@param {type:"string"}
MEMO_TEXT = "test" #@param {type:"string"}
BASE_FEE = 10000 #@param {type:"integer"}
SET_TIMEOUT = 30 #@param {type:"integer"}

if (not SECRET_KEY or not ACCOUNT_CSV_PATH or not MEMO_TEXT or not BASE_FEE or not SET_TIMEOUT):
  print('Please fill out correct value')
  raise

from IPython.display import clear_output
!pip install stellar-sdk==8.1.1 urllib3==1.26.13
clear_output()
import argparse
import datetime
import requests
import pandas as pd
from stellar_sdk import Asset, Claimant, ClaimPredicate, Keypair, Network, Server, TransactionBuilder

def get_time():
    return datetime.datetime.now().strftime("%d%m%Y-%I:%M:%S%p")


def load_accounts(path):
    data = pd.read_csv(path)

    if sum(data.isnull().sum().to_list()) != 0:
        raise Exception('Please remove null row in this csv')

    return data


def source_account(source_secret_key):
    source_keypair = Keypair.from_secret(source_secret_key)
    source_public_key = source_keypair.public_key
    source_account_getted = server.load_account(source_public_key)
    return (source_account_getted, source_keypair)


def send_token(transaction, source_keypair):

    transaction.sign(source_keypair)

    response = server.submit_transaction(transaction)
    return response


def newTrcBuil(source_account_getted, memo):
    return TransactionBuilder(
        source_account=source_account_getted,
        network_passphrase=Network.PUBLIC_NETWORK_PASSPHRASE,
        base_fee=BASE_FEE,
    ).add_text_memo(memo)

def makePayment(
    assetCode, assetIssuer, amount, 
    list_claimants, source_keypair, 
    source_account_getted, no):
    p = newTrcBuil(source_account_getted, MEMO_TEXT)
    p.append_create_claimable_balance_op(
        Asset(
            assetCode,
            assetIssuer,
        ), amount,
        list_claimants,
        source=source_keypair.public_key
    )
    list_claimants = []
    transaction = (
        p
        .set_timeout(SET_TIMEOUT)
        .build()
    )
    send_return = send_token(transaction, source_keypair)
    trxLink = send_return["_links"]['self']['href']
    print_log = f'{get_time()} {no+1:5d} {"Successful" if send_return["successful"] else "Unsuccessful"} {trxLink} {send_return["source_account"]} '
    print(print_log)
    destitaion_account = requests.get(
        send_return['_links']['operations']['href'][:-21], params={"limit": 200}).json()['_embedded']['records']
    print_log = print_log[:-1]+"\n"
    with open("history.txt", "a+") as w:
        w.write(print_log)

if __name__ == "__main__":
    # parser = argparse.ArgumentParser(
    #     description='Stellar: Auto send pay to your defined list')
    # parser.add_argument(
    #     '-ssk', type=str, help='Your source secret key.', required=True)
    # parser.add_argument(
    #     '-rlist', type=str, help='Destination public key, file path.', required=True)
    # parser.add_argument('-m', type=str, help='Memo massage', required=True)
    # args = parser.parse_args()
    

    server = Server(horizon_url="https://horizon.stellar.org")
    sa = source_account(SECRET_KEY)
    source_account_getted, source_keypair = sa
    account_list = load_accounts(ACCOUNT_CSV_PATH)

    list_claimants = []
    lenAccounts = len(account_list[:5])
    for no in range(lenAccounts):
        list_claimants.append(
            Claimant(
                destination=account_list['Address'][no],
                predicate=ClaimPredicate.predicate_not(
                    ClaimPredicate.predicate_before_relative_time(1209600))
            ))
        if len(list_claimants) >= 2:
            makePayment(account_list['Asset code'][no],
                account_list['Asset issuer'][no],
                str(account_list['Amount'][no]),
                list_claimants, source_keypair,
                source_account_getted, no
            )
            list_claimants = []
    
    if list_claimants:
        lastOne = lenAccounts-1
        makePayment(account_list['Asset code'][lastOne],
            account_list['Asset issuer'][lastOne],
            str(account_list['Amount'][lastOne]),
            list_claimants, source_keypair,
            source_account_getted, lastOne
        )